In [4]:
import pandas as pd 
import numpy as np 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
from datetime import date


In [5]:

today = date.today()
d1 = today.strftime("%d/%m/%Y")


In [16]:
dataset = pd.read_csv('../airflow/raw/scrapping_anuncios_shopcar.csv', encoding='latin') 
dataset.columns = ['Unnamed: 0', 'Modelo', 'Ano', 'Cor', 'Combustivel', 'KM', 'Preco', 'Link', 'Vendedor', 'Cidade']
# ## geração de id do shop car

dataset['Id_Anuncio_ShopCar'] = pd.Series(dataset['Link']).str.slice(start=-7)
dataset.reset_index()
dataset.set_index('Id_Anuncio_ShopCar', inplace=True)
dataset.drop(labels=['Unnamed: 0'], axis=1)

## correção do campo data ano/modelo

ano_fabricacao = pd.Series(dataset['Ano']).str.slice(stop=2)
dataset['Ano_Fabricacao'] = pd.to_numeric(ano_fabricacao)
dataset['Ano_Fabricacao'] = np.where(dataset['Ano_Fabricacao'] < 22, 2000 + dataset['Ano_Fabricacao'], 1900 + dataset['Ano_Fabricacao'])

ano_modelo = pd.Series(dataset['Ano']).str.slice(start=3, stop=5)
dataset['Ano_Modelo'] = pd.to_numeric(ano_modelo)
dataset['Ano_Modelo'] = np.where(dataset['Ano_Modelo'] < 22, 2000 + dataset['Ano_Modelo'], 1900 + dataset['Ano_Modelo'])
dataset = dataset.drop(labels=['Ano'], axis=1)
dataset = dataset.drop(labels=['Unnamed: 0'], axis=1)

## correção do campo KM
dataset['KM'] = dataset['KM'].str.replace('Km', '')
dataset['KM'] = dataset['KM'].str.replace('.', '')
dataset['KM'] = pd.to_numeric(dataset['KM'], errors='coerce')

## correção do campo KM
dataset['Preco'] = dataset['Preco'].str.replace('R', '')
dataset['Preco'] = dataset['Preco'].str.replace('$', '')
dataset['Preco'] = dataset['Preco'].str.replace('.', '')
dataset['Preco'] = dataset['Preco'].str.replace(',00', '')
dataset['Preco'] = pd.to_numeric(dataset['Preco'], errors='coerce')

## endereços 
enderecos = pd.Series(dataset['Cidade'])
dataset['Estado_Anuncio'] = enderecos.str.partition('/')[2]
dataset['Endereco_Anuncio'] = enderecos.str.partition('-')[0]

enderecos = enderecos.str.partition('-')[2]
enderecos.str.partition(' - ')
dataset['Bairro_Anuncio'] = enderecos.str.partition('-')[0]

enderecos = enderecos.str.partition('-')[2]
enderecos = enderecos.str.partition('/')
dataset['Cidade_Anuncio'] = enderecos[0]

## marca 
marca = dataset['Link'].str.partition('/')[2]
dataset['Marca'] = marca.str.partition('/')[2].str.partition('/')[2].str.partition('/')[2].str.partition('/')[0]
dataset['Data_Extracao_Dados'] = d1

dataset


C:\Users\mayco\AppData\Local\Temp/ipykernel_3300/1299740752.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataset['KM'] = dataset['KM'].str.replace('.', '')
C:\Users\mayco\AppData\Local\Temp/ipykernel_3300/1299740752.py:29: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataset['Preco'] = dataset['Preco'].str.replace('$', '')
C:\Users\mayco\AppData\Local\Temp/ipykernel_3300/1299740752.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataset['Preco'] = dataset['Preco'].str.replace('.', '')


,Modelo,Cor,Combustivel,KM,Preco,Link,Vendedor,Cidade,Ano_Fabricacao,Ano_Modelo,Estado_Anuncio,Endereco_Anuncio,Bairro_Anuncio,Cidade_Anuncio,Marca,Data_Extracao_Dados
Id_Anuncio_ShopCar,,,,,,,,,,,,,,,,
1114104,Xsara Picasso GLX 1.6 16v,Preta,Flex,125000.0,21800,https://www.shopcar.com.br/veiculos/citro-n/xs...,Premium Motors,"Rua João Pedro de Souza,73 - Jardim Monte Líba...",2010,2011,MS,"Rua João Pedro de Souza,73",Jardim Monte Líbano,Campo Grande,citro-n,03/12/2021
1112370,Logan Expression 1.6,Prata,Flex,NaN,21990,https://www.shopcar.com.br/veiculos/renault/lo...,KV Veículos,"R. Alegrete, 2139 - Cel. Antonino - Campo Gran...",2008,2009,MS,"R. Alegrete, 2139",Cel. Antonino,Campo Grande,renault,03/12/2021
1135084,Fiesta Hatch 1.0 4p.,Preta,Flex,105000.0,22500,https://www.shopcar.com.br/veiculos/ford/fiest...,Amaral Motors,"Av. Pres. Ernesto Geisel, 5701 - Cabreúva - Ca...",2010,2011,MS,"Av. Pres. Ernesto Geisel, 5701",Cabreúva,Campo Grande,ford,03/12/2021
1127771,Fox 1.0 4p.,Preta,Flex,120000.0,23000,https://www.shopcar.com.br/veiculos/vw-volkswa...,Prime Veículos,"Av. Pres. Ernesto Geisel, 5721 - Cabreúva - Ca...",2009,2010,MS,"Av. Pres. Ernesto Geisel, 5721",Cabreúva,Campo Grande,vw-volkswagen,03/12/2021
1130876,Uno Mille Economy Way 1.0 Fire 2p.,Branca,Flex,115000.0,23000,https://www.shopcar.com.br/veiculos/fiat/uno-m...,João Pedro Dias,Ribas do Rio Pardo - MS,2010,2010,,Ribas do Rio Pardo,MS,,fiat,03/12/2021
1135240,Astra Sedan Elegance 2.0 A/T,Prata,Flex,157194.0,23500,https://www.shopcar.com.br/veiculos/gm-chevrol...,Glauco Renato Azevedo Teixeira,Campo Grande - MS,2007,2008,,Campo Grande,MS,,gm-chevrolet,03/12/2021
1128056,Palio Celebration 1.0 4p.,Preta,Flex,118000.0,23500,https://www.shopcar.com.br/veiculos/fiat/palio...,Águia Veículos,"Av. Bandeirantes, 1333 - Taquarussu - Campo Gr...",2009,2010,MS,"Av. Bandeirantes, 1333",Taquarussu,Campo Grande,fiat,03/12/2021
1133336,207 Passion XS 1.6 16v,Prata,Flex,87000.0,23800,https://www.shopcar.com.br/veiculos/peugeot/20...,Montana Veículos,"Rua José Antonio, 114 - Vila Rosa Pires - Camp...",2011,2012,MS,"Rua José Antonio, 114",Vila Rosa Pires,Campo Grande,peugeot,03/12/2021
1136086,Fiesta Hatch 1.0 4p.,Preta,Flex,NaN,23900,https://www.shopcar.com.br/veiculos/ford/fiest...,Premium Motors,"Rua João Pedro de Souza,73 - Jardim Monte Líba...",2011,2011,MS,"Rua João Pedro de Souza,73",Jardim Monte Líbano,Campo Grande,ford,03/12/2021


In [ ]:
filtro = dataset_final[(dataset_final.Preco < 40000) & (dataset_final.KM < 50000)].sort_values(by = 'Preco')

In [ ]:
qtd_veiculos = dataset.Modelo.count()
qtd_veiculos2 = filtro.Modelo.count()

In [ ]:

email_user = ''
email_password = ''

recipients = ['mayconfelipemotamw3@gmail.com', 'jorgematosltda@gmail.com', 'jvalencarmg@gmail.com'] 
emaillist = [elem.strip().split(',') for elem in recipients]
msg = MIMEMultipart()
msg['Subject'] = f'|SHOP CAR & WebMotors| - {dataset.Modelo.count() + dataset_wm.Modelo.count()} carros extraídos com sucesso! {qtd_veiculos2} parecem ser bons!'
msg['From'] = 'maycon.mota@gmail.com'


html = """\
<html>
  <head></head>
  <body>
<div>
Bom dia, encaminho o relatório contendo todos os veículos do www.shopcar.com.br e www.webmotors.com.br, extraídos na data de {0}. São filtrados veículos com KM menor que 50000 e menos de R$ 50000. <br>
Quantidade de extrações do ShopCar: {1} <br>
Quantidade de extrações do WebMotors: {2} <br>
Estatística descritiva: {3} <br>
Boa sorte!
</div>
    {4}
  </body>
</html>
""".format(d1, dataset.Modelo.count(), dataset_wm.Modelo.count(), filtro.describe().to_html(), filtro.to_html())

part1 = MIMEText(html, 'html')
msg.attach(part1)

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(email_user,email_password)
server.sendmail(msg['From'], emaillist , msg.as_string())

In [ ]:
filtro